In [1]:
print("Test YaYaGenPE")

In [2]:
import pandas as pd
import operator
import ast
import os
from shutil import copyfile
import shutil

In [3]:
df = pd.read_csv("rules_fixed.csv", index_col=0)
df.head()

,Exe name,Family,first_seen,last_seen,total,positives,ssdeep,imphash,yara_detected,yara_matching_rules
0,4bf4bc2c3309458670687f7f35a034e87726c0b2b4ce59...,teslacrypt,2016-03-22 05:29:56,2016-07-19 11:25:46,57.0,47.0,6144:bZ6bJnUPNKASkonBh2H5um+WoqhRXysUHPBBSS:bZ...,8f674a2532aeb9bb895ceb2276f96e82,True,['../yara/rules-master/malware/VectraThreatLab...
1,e92da7e044e089f5793e7458abedc01ab2f9fabdc26c12...,zerber,2017-05-12 05:45:03,2017-09-25 09:45:31,65.0,53.0,6144:Xb726fs3RPPsT5lw1MkUuPW+NsRjscQSj7PrTBFTT...,3de01d3ecac524c6e75df7762c12aff1,True,['../yara/rules-master/malware/VectraThreatLab...
2,6cd351142fadc8798cec7a4949fed55b14cb2f779082cc...,zerber,2017-05-21 15:07:48,2017-05-21 15:07:48,62.0,51.0,6144:Cw0VTtcjjXjloKH4RXGkMbH653yAehEOvzaKF/8iF...,f9c7630226d1cef8ca9f2a61f77c9e32,True,['../yara/rules-master/malware/VectraThreatLab...
3,bb37f3f511683bb363e6fe62a2e83fc74b447646e1697d...,razy,2017-02-04 11:03:59,2017-02-04 11:03:59,56.0,31.0,3072:jwsBQaaZH40gr+QdfPNzkV8a7DjHWbGRq58Onla:/...,4e58119b2e3ad35755948eea16c3854f,False,[]
4,c64b6a13b20a3f9c6f8dcf2af17d974eb104b2ff98eec3...,teslacrypt,2016-03-18 07:40:14,2016-03-24 12:37:34,57.0,48.0,6144:ia6t9pGTFyeiubGw74fyIMrm2GAtJtgMOJcFyw+Yf...,b0549bcf98dfeef5fe0d7037087e1e35,True,['../yara/rules-master/malware/VectraThreatLab...


In [4]:
df_tesla = df[df['Family'] == 'teslacrypt']

In [5]:
# # Create a dict to count the rule match frequency
# ruleset = dict()
# for v in df_tesla.yara_matching_rules.values:
#     v = ast.literal_eval(v)
#     for r in v:
#         if r in ruleset:
#             ruleset[r] += 1
#         else:
#             ruleset[r] = 1

# # Sort the results
# ruleset_sorted = sorted(ruleset.items(), key=operator.itemgetter(1))
# for k in ruleset_sorted:
#     print(k[1], k[0])

In [6]:
# Prendi solo i sample che matchano questa regola
rr = '../yara/rules-master/malware/Icewater/Icewater-master/n3ed/n3ed.291771a8c2000b32.yara:n3ed_291771a8c2000b32'

sha256s = list()
for c, v in enumerate(df_tesla.yara_matching_rules.values):
    if rr in v:
        sha256s.append(df_tesla.iloc[c]['Exe name'])
print(len(sha256s))

## Altrimenti prendi tutti
# sha256s = list(df_tesla['Exe name'].values)

484


In [7]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=20, shuffle=True)
print("Num of splits: %d" % kf.get_n_splits(sha256s))
splits = list(kf.split(sha256s))

# Be aware that train and test are inverted in respect to the ML terminology. Here few samples are enough to create a rule
train_indexes = splits[0][1]
test_indexes = splits[0][0] 
print("Len train index set: %d" % len(train_indexes))
print("Let test index set: %d" % len(test_indexes))

train = [sha256s[i] for i in train_indexes]
test = [sha256s[i] for i in test_indexes]
print("Len train set: %d" % len(train))
print("Let test set: %d" % len(test))
# print(train)
print(train)

Num of splits: 20
Len train index set: 25
Let test index set: 459
Len train set: 25
Let test set: 459
['71b0a027a737fc78be23cb18bb9c10cf3cb5cc2b340e7572eef2ede2a0e4a4e9', 'e18cef6f57f576d366000092cce3355b34f56e891cea619f885f75af48d4f5be', '1eddbdf94fca23356d5853551cd6ab3dcffae6f57fe83b164646ed6584fb4eee', '2924b79af4bdbefdfbdcbcb2df65a0fa7c686edd76fc414afa0fb63b44218762', 'cc8cb9d9c7ea691e14572490ff5c258c5ce9267d5748883e64f951cae3af5d8c', 'b0b523ce0cc39125f7209aa8b27ae1a83b7dc769fc09ddbf6b721b3f005f479e', 'f1982bf38a0f43eb3273ec5ec2852bcb082e8eed543c555eb0ff5e239e4ceb73', '86ded8b77b52891bf8680dea86d7ae99edf34357e9d50012cd712f930bc5e452', '301632853bc391b903897cd76b2135fe8b2607d49368a56c982c313580f93040', 'e3e024e68d2d3d499a609d7d3d06ed76ccbb6cff2bf7937f401abea2c743f1d4', '7fc5e17ad9de79a9bbcf20f4a07dec7e0c3823877ec60fd1970b15729ef30a4d', '17c42ee970fec346f2d83aa7b1d156c33a634a1db4dc41badfd33d842f0ff8f0', '6499852e8026ff4e88a8d664994bbb3aec6552078d57b10d600d5e4fdd15314b', '97b28d3fdadf

In [8]:
print("Copying training dataset")

if os.path.isdir("tesla_train"):
    shutil.rmtree('tesla_train/')
os.mkdir("tesla_train")

for sha in train:
    copyfile("Win32_EXE/%s" % sha, "tesla_train/%s" % sha)


print("Copying test dataset")

if os.path.isdir("tesla_test"):
    shutil.rmtree('tesla_test/')
os.mkdir("tesla_test")

for sha in test:
    copyfile("Win32_EXE/%s" % sha, "tesla_test/%s" % sha)

Copying training dataset
Copying test dataset


In [9]:
if os.path.isdir("/Users/jimmy/Documents/_Research/_Tesi/Luca Cetro/github/YaYaGenPE/output_tesla"):
    shutil.rmtree("/Users/jimmy/Documents/_Research/_Tesi/Luca Cetro/github/YaYaGenPE/output_tesla")

! cd "/Users/jimmy/Documents/_Research/_Tesi/Luca Cetro/github/YaYaGenPE"; ./yyg.py -ndb -a greedy -c _config/configuration.json -dir /Users/jimmy/Desktop/tesla_train/ -o output_tesla/


__  __   __  __     _____         ___  ____ 
\ \/ /__ \ \/ /__ _/ ___/__ ___  / _ \/ __/  YaYaGenPE -- Yet Another Yara Rule Generator for PE files
 \  / _ `/\  / _ `/ (_ / -_) _ \/ ___/ _/    (!) v0.1_spring18
 /_/\_,_/ /_/\_,_/\___/\__/_//_/_/  /___/    by Andrea Marcelli & Giovanni Squillero & Luca Cetro

12:05:50 WARNING:: Let's start!
 
12:05:50 INFO:: Processing: /Users/jimmy/Desktop/tesla_train/f1982bf38a0f43eb3273ec5ec2852bcb082e8eed543c555eb0ff5e239e4ceb73
12:05:51 WARNING:: Skipping EXPORT DIRECTORY
 
12:05:51 INFO:: Processing: /Users/jimmy/Desktop/tesla_train/478c87f9efb311b80eec019ac1a1075568ce3e222cc9529e8c4f83c6721678c2
 
12:05:51 INFO:: Processing: /Users/jimmy/Desktop/tesla_train/def73d87e78b370ef9a88ff83d37d8887db31b1ad08ebd4c36731211b9a035df
 
12:05:51 INFO:: Processing: /Users/jimmy/Desktop/tesla_train/49a63c4c4b6e0ac5789de433e2a510347aee0ec33eac0c19852f031785df5574
 
12:05:51 INFO:: Processing: /Users/jimmy/Desktop/tesla_train/eb1ed60e0664dbc9775e3d898d3dadc428f56

In [10]:
if os.path.isdir("/Users/jimmy/Documents/_Research/_Tesi/Luca Cetro/github/YaYaGenPE/output_tesla_check"):
    shutil.rmtree("/Users/jimmy/Documents/_Research/_Tesi/Luca Cetro/github/YaYaGenPE/output_tesla_check")
os.mkdir("/Users/jimmy/Documents/_Research/_Tesi/Luca Cetro/github/YaYaGenPE/output_tesla_check")

! cd "/Users/jimmy/Documents/_Research/_Tesi/Luca Cetro/github/YaYaGenPE"; ./check_yara.py -nc 4 -i empty_file -r /Users/jimmy/Desktop/tesla_test/ -o output_tesla_check/ output_tesla/*.yar

12:05:56 WARNING:: Let's start!
12:05:56 INFO:: Compiling YARA ruleset...
12:05:56 INFO:: Loading yara_ruleset...
12:05:56 INFO:: Initial ruleset compilation...
12:05:56 INFO:: Compiled 1/1 rules
12:05:56 INFO:: Final ruleset compilation...
12:05:56 INFO:: Ruleset compilation completed
12:05:56 INFO:: Loading sha256 list...
12:05:56 INFO:: Starting 4 consumers
12:05:56 INFO:: Started checking...
12:05:59 INFO:: Waiting for consumers to join
12:05:59 INFO:: Collecting results from consumers
12:05:59 INFO:: PE files checked: 459
12:05:59 INFO:: PE files skipped: 0
12:05:59 INFO:: Len TP list: 0
12:05:59 INFO:: TP list matches: 0
12:05:59 INFO:: Checking 459 pefiles took 3.48 seconds
12:05:59 WARNING:: That's all folks


In [11]:
df_result = pd.read_csv("/Users/jimmy/Documents/_Research/_Tesi/Luca Cetro/github/YaYaGenPE/output_tesla_check/sha256_pandas.csv", index_col=0)

In [12]:
df_result.head()
df_result.values
c = 0
for x in df_result.values:
    if sum(x) > 0:
        c += 1
print("Total samples: %d" % len(df_result.values))
print("Total matches: %d" % c)

Total samples: 459
Total matches: 413


In [13]:
df_result.head()

,rule_0.yar
a2dc35392fcddcb4a3b422d2b94062557330467d5deeef23c0fda4c62fbcbfb1,1
572c6bf209c79c478178c6d84e2e630019b4ab33f4838848349e37baf34921f5,1
44e2eebc0eccc30ca1d7121eacaad4f734e29ab118d49e3628aa75aaf46f0a2e,1
28fb92db972d25c8e5575e40f5357c7e9736f233432e97ee7e4369bd9a4ef38a,1
861293295735bf3a518843409fb226968f9d6e20f8b715a2a250e5dba13467a3,1
